In [6]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('Customers.csv', 'Customers'),
    ('Data_Dictionary.csv', 'Data_Dictionary'),
    ('Exchange_Rates.csv', 'Exchange_Rates'),
    ('Products.csv', 'Products'),
    ('Sales.csv', 'Sales'),
    ('Stores.csv', 'Stores')
]

# Connect to the MySQL database
try:
    conn = mysql.connector.connect(
        host='127.0.0.1',
        port=3306,
        user='root',
        password='595162sushiielA@',
        database='project22'
    )
    cursor = conn.cursor()
except mysql.connector.Error as err:
    print(f"Error connecting to MySQL: {err}")
    exit(1)

# Folder containing the CSV files
folder_path = 'C:/Users/Hp/Desktop/Project1'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

def read_csv_with_encodings(file_path, encodings=['utf-8', 'ISO-8859-1', 'Windows-1252']):
    for encoding in encodings:
        try:
            return pd.read_csv(file_path, encoding=encoding)
        except UnicodeDecodeError:
            print(f"Failed to read {file_path} with encoding {encoding}. Trying next encoding...")
    raise ValueError(f"Unable to read {file_path} with any of the specified encodings.")

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    try:
        # Read the CSV file into a pandas DataFrame with different encodings
        df = read_csv_with_encodings(file_path)
    except Exception as e:
        print(f"Failed to read {csv_file}: {e}")
        continue  # Skip to the next file

    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)

    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    try:
        # Create a list of tuples for batch insert
        data_tuples = [
            tuple(None if pd.isna(x) else x for x in row) for row in df.itertuples(index=False)
        ]
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(df.columns))})"
        cursor.executemany(sql, data_tuples)

        # Commit the transaction for the current CSV file
        conn.commit()
    except Exception as e:
        print(f"Failed to insert data for {csv_file}: {e}")

# Close the connection
cursor.close()
conn.close()


Failed to read C:/Users/Hp/Desktop/Project1\Customers.csv with encoding utf-8. Trying next encoding...
Processing Customers.csv
NaN values before replacement:
CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

Processing Data_Dictionary.csv
NaN values before replacement:
Table          0
Field          0
Description    0
dtype: int64

Processing Exchange_Rates.csv
NaN values before replacement:
Date        0
Currency    0
Exchange    0
dtype: int64

Processing Products.csv
NaN values before replacement:
ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64

Processing Sales.csv
NaN values before replacement:
Order Number         0
Line Item            0
Order Date           0
Delivery Date

In [7]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


What types of products does the company sell, and where are customers located?

Are there any seasonal patterns or trends for order volume or revenue?

How long is the average delivery time in days? Has that changed over time?

Is there a difference in average order value (AOV) for online vs. in-store sales

In [9]:
# Question 1: Types of Products and Customer Locations
# 1a. Product Types
import pandas as pd
product_types_query = """
SELECT DISTINCT Subcategory, COUNT(Product_Name) AS Product_Count
FROM Products
GROUP BY Subcategory;
"""
product_types = pd.read_sql_query(product_types_query, conn)
print("Product Types:\n", product_types)

C:\Users\Hp\AppData\Local\Temp\ipykernel_12276\1751872455.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_types = pd.read_sql_query(product_types_query, conn)


OperationalError: MySQL Connection not available

In [ ]:
# 1b. Customer Locations
customer_locations_query = """
SELECT DISTINCT City || ' ' || State || ' ' || Country AS Customer_Location
FROM Customers;
"""
customer_locations = pd.read_sql_query(customer_locations_query, conn)
print("Customer Locations:\n", customer_locations)



In [ ]:
# Question 2: Monthly and Quarterly Order Volumes and Revenue Trends
# Add `Order_con` and `Del_con` to Sales to work with datetime columns in Pandas
sales_df['Order_con'] = pd.to_datetime(sales_df['Order_Date'], format='%m/%d/%Y')
sales_df['Del_con'] = pd.to_datetime(sales_df['Delivery_Date'], format='%m/%d/%Y')
sales_df.to_sql('converted', conn, index=False, if_exists='replace')

# Monthly Order Volume and Revenue
monthly_trends_query = """
SELECT strftime('%Y-%m', Order_con) AS Order_Month, 
       COUNT(Order_Number) AS Total_Orders, 
       SUM(Revenue) AS Total_Revenue
FROM converted
GROUP BY Order_Month;
"""
monthly_trends = pd.read_sql_query(monthly_trends_query, conn)
print("Monthly Order Volumes and Revenue:\n", monthly_trends)

# Quarterly Order Volume and Revenue
quarterly_trends_query = """
SELECT strftime('%Y-Q%Y', Order_con) AS Order_Quarter, 
       COUNT(Order_Number) AS Total_Orders, 
       SUM(Revenue) AS Total_Revenue
FROM converted
GROUP BY Order_Quarter;
"""
quarterly_trends = pd.read_sql_query(quarterly_trends_query, conn)
print("Quarterly Order Volumes and Revenue:\n", quarterly_trends)



In [ ]:
# Question 3: Average Delivery Time
delivery_time_query = """
SELECT AVG(julianday(Del_con) - julianday(Order_con)) AS Average_Delivery_Time
FROM converted
WHERE Del_con IS NOT NULL;
"""
average_delivery_time = pd.read_sql_query(delivery_time_query, conn)
print("Average Delivery Time (days):\n", average_delivery_time)



In [ ]:
# Question 4: Seasonal Trends for Order Volume and Revenue
# Use Pandas for advanced time-based analysis
sales_df['Order_Month'] = sales_df['Order_con'].dt.to_period("M")
monthly_orders = sales_df.groupby('Order_Month')['Order_Number'].count()
monthly_revenue = sales_df.groupby('Order_Month')['Revenue'].sum()
print("Monthly Order Volume and Revenue Trends:\n", pd.DataFrame({'Orders': monthly_orders, 'Revenue': monthly_revenue}))



In [ ]:
# Question 5: Most Common Products by Region
# Join Customers and Sales data to analyze by region
common_products_region_query = """
SELECT c.Country, c.State, c.City, p.Subcategory, COUNT(s.Order_Number) AS Order_Count
FROM Sales s
JOIN Customers c ON s.Customer_ID = c.Customer_ID
JOIN Products p ON s.Product_ID = p.Product_ID
GROUP BY c.Country, c.State, c.City, p.Subcategory
ORDER BY Order_Count DESC;
"""
common_products_region = pd.read_sql_query(common_products_region_query, conn)
print("Most Common Products by Region:\n", common_products_region)

# Close the database connection
conn.close()